# Integração e pré-processamento das bases de dados de jogos eletrônicos

Para esta tarefa, utilizaremos somente as biblioteca `pandas`, `numpy` e `scikitlearn`. 

In [168]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

df1 = pd.read_csv('trabalho2_dados_1.csv')
df2 = pd.read_csv('trabalho3_dados_adicionais_1.csv')
df = pd.concat([df1, df2])

print("Total de jogos na base de dados 1: {}".format(len(df1)))
print("Total de jogos na base de dados 2: {}".format(len(df2)))

print("\nBase de dados resultante da operação de concatenação:")

df

Total de jogos na base de dados 1: 1632
Total de jogos na base de dados 2: 1588

Base de dados resultante da operação de concatenação:


,nome,plataforma,genero,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante
0,Jelly Belly: Ballistic Beans,Wii,Puzzle,Zoo Digital Publishing,0.02,21-Apr-09,NaN,NaN,tbd,NaN,Zoo Digital Publishing
1,Madden NFL 12,PSP,Sports,Electronic Arts,0.20,30-Aug-11,NaN,NaN,tbd,NaN,EA Tiburon
2,The Sims 2: Pets,Wii,Simulation,Electronic Arts,0.46,12-Jun-07,65.0,15.0,tbd,NaN,Maxis
3,Guilty Gear XX Accent Core Plus,PSP,Fighting,PQube,0.08,7-Apr-09,NaN,NaN,8.3,4.0,Arc System Works
4,WWE 2K14,PS3,Sports,Take-Two Interactive,0.72,29-Oct-13,74.0,21.0,7.4,60.0,Yuke's
...,...,...,...,...,...,...,...,...,...,...,...
1583,Major League Baseball 2K8,PSP,Sports,Unknown,0.03,3-Mar-08,63.0,5.0,tbd,NaN,Kush Games
1584,Monster Hunter Frontier Online,PS3,Role-Playing,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN
1585,Battle vs. Chess,PS3,Misc,TopWare Interactive,0.03,TBA,NaN,NaN,NaN,NaN,TopWare Interactive
1586,AKB1/48: Idol to Guam de Koishitara...,X360,Misc,NaN,0.01,NaN,NaN,NaN,NaN,NaN,NaN


## Tratamento de variáveis categóricas

Há somente dois atributos categóricos: **plataforma** e **gênero**.

In [169]:
# Atributos categóricos
df['plataforma'] = df['plataforma'].astype('category')
df['genero'] = df['genero'].astype('category')

print("Plataformas: {}".format(df['plataforma'].cat.categories))
print("Gêneros: {}".format(df['genero'].cat.categories))

Plataformas: Index(['3DS', 'PS3', 'PS4', 'PSP', 'PSV', 'Wii', 'WiiU', 'X360', 'XOne'], dtype='object')
Gêneros: Index(['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
       'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports',
       'Strategy'],
      dtype='object')


## Unicidade

O nome do jogo eletrônico atua como chave-primária, portanto é necessário remover os jogos que aparecem repetidos no *dataframe*. Para manter a indexação consistente, utiliza-se a função `reset_index`.

In [170]:
# Jogos são únicos
df = df.drop_duplicates(subset = ['nome'])
df = df.reset_index(drop = True)

df[['nome']]

,nome
0,Jelly Belly: Ballistic Beans
1,Madden NFL 12
2,The Sims 2: Pets
3,Guilty Gear XX Accent Core Plus
4,WWE 2K14
...,...
1442,D.C. III: Da Capo III
1443,Street Fighter X Tekken
1444,John Daly's ProStroke Golf
1445,Backyard Sports Football: Rookie Rush


## Tratamento de dados faltantes

Foram identificados os seguintes atributos faltantes em algumas instâncias do *dataframe*:

In [171]:
df.isnull().sum()

nome                    0
plataforma              0
genero                  0
editora                 4
vendas                  0
lancamento            455
avaliacao-criticos    639
numero-criticos       639
avaliacao-usuarios    470
numero-usuarios       693
fabricante            459
dtype: int64

Neste momento, há somente preocupação em tratar dados **numéricos** faltantes, uma vez que estes são os mais sensíveis para a aplicação dos métodos de aprendizado de máquina:

*  avaliacao-criticos
*  numero-criticos
*  avaliacao-usuarios
*  numero-usuarios

O preenchimento de dados faltantes será feito através da média geral do gênero do jogo em questão. 

In [172]:
# Converter valores 'tbd' para NaN
df['avaliacao-usuarios'] = pd.to_numeric(df['avaliacao-usuarios'], errors = 'coerce')

# Agrupando por gênero e coletando a média para os valores faltantes
df['avaliacao-criticos'] = df.groupby('genero')['avaliacao-criticos'].transform(lambda x: x.fillna(x.mean()))
df['avaliacao-usuarios'] = df.groupby('genero')['avaliacao-usuarios'].transform(lambda x: x.fillna(x.mean()))
df['numero-criticos'] = df.groupby('genero')['numero-criticos'].transform(lambda x: x.fillna(x.mean())).astype('int')
df['numero-usuarios'] = df.groupby('genero')['numero-usuarios'].transform(lambda x: x.fillna(x.mean())).astype('int')

df[['nome', 'genero', 'avaliacao-criticos', 'avaliacao-usuarios', 'numero-criticos', 'numero-usuarios']]

,nome,genero,avaliacao-criticos,avaliacao-usuarios,numero-criticos,numero-usuarios
0,Jelly Belly: Ballistic Beans,Puzzle,62.095238,6.133333,17,10
1,Madden NFL 12,Sports,66.848485,6.804167,22,29
2,The Sims 2: Pets,Simulation,65.000000,7.558333,15,31
3,Guilty Gear XX Accent Core Plus,Fighting,70.160000,8.300000,30,4
4,WWE 2K14,Sports,74.000000,7.400000,21,60
...,...,...,...,...,...,...
1442,D.C. III: Da Capo III,Adventure,67.439024,7.119048,32,331
1443,Street Fighter X Tekken,Fighting,83.000000,4.200000,45,162
1444,John Daly's ProStroke Golf,Sports,57.000000,6.804167,5,29
1445,Backyard Sports Football: Rookie Rush,Sports,66.848485,6.804167,22,29


## Normalização dos dados

*  Para que métodos de aprendizado de máquina operem melhor sobre atributos categóricos, utiliza-se o método `get_dummies()` para gerar um atributo binário para cada categoria existente. 
*  Além disso, todos os dados númericos foram normalizados no intervalo \[0, 1\] utilizando o scaler `MinMaxScaler`.


In [173]:
# Similar ao LabelBinarizer
df = pd.get_dummies(df, columns = ['plataforma', 'genero'])

df[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios']] = MinMaxScaler().fit_transform(df[['vendas', 'avaliacao-criticos', 'numero-criticos', 'avaliacao-usuarios', 'numero-usuarios']])

df

,nome,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante,plataforma_3DS,...,genero_Fighting,genero_Misc,genero_Platform,genero_Puzzle,genero_Racing,genero_Role-Playing,genero_Shooter,genero_Simulation,genero_Sports,genero_Strategy
0,Jelly Belly: Ballistic Beans,Zoo Digital Publishing,0.000122,21-Apr-09,0.577591,0.126214,0.627451,0.000710,Zoo Digital Publishing,0,...,0,0,0,1,0,0,0,0,0,0
1,Madden NFL 12,Electronic Arts,0.002322,30-Aug-11,0.633512,0.174757,0.706373,0.002958,EA Tiburon,0,...,0,0,0,0,0,0,0,0,1,0
2,The Sims 2: Pets,Electronic Arts,0.005499,12-Jun-07,0.611765,0.106796,0.795098,0.003194,Maxis,0,...,0,0,0,0,0,0,0,1,0,0
3,Guilty Gear XX Accent Core Plus,PQube,0.000855,7-Apr-09,0.672471,0.252427,0.882353,0.000000,Arc System Works,0,...,1,0,0,0,0,0,0,0,0,0
4,WWE 2K14,Take-Two Interactive,0.008677,29-Oct-13,0.717647,0.165049,0.776471,0.006625,Yuke's,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442,D.C. III: Da Capo III,Kadokawa Games,0.000244,NaN,0.640459,0.271845,0.743417,0.038684,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1443,Street Fighter X Tekken,Capcom,0.004644,6-Mar-12,0.823529,0.398058,0.400000,0.018692,Capcom,0,...,1,0,0,0,0,0,0,0,0,0
1444,John Daly's ProStroke Golf,O-Games,0.000000,Canceled,0.517647,0.009709,0.706373,0.002958,Gusto Games,0,...,0,0,0,0,0,0,0,0,1,0
1445,Backyard Sports Football: Rookie Rush,Atari,0.000855,NaN,0.633512,0.174757,0.706373,0.002958,NaN,0,...,0,0,0,0,0,0,0,0,1,0


## Detecção de outliers

Para esta base de dados, considerou-se como **outliers** instâncias que apresentam valores extremos (para mais ou para menos) no atributo **vendas**, com intervalo de confiança de 95%.
O motivo para esta decisão vem do fato de que, além de ser um atributo importante, todas as instâncias continham um valor inicial daquele, não necessitando de preenchimento artificial. 



In [174]:
df = df[np.abs(df['vendas'] - df['vendas'].mean()) <= 1.96 * df['vendas'].std()]

df

,nome,editora,vendas,lancamento,avaliacao-criticos,numero-criticos,avaliacao-usuarios,numero-usuarios,fabricante,plataforma_3DS,...,genero_Fighting,genero_Misc,genero_Platform,genero_Puzzle,genero_Racing,genero_Role-Playing,genero_Shooter,genero_Simulation,genero_Sports,genero_Strategy
0,Jelly Belly: Ballistic Beans,Zoo Digital Publishing,0.000122,21-Apr-09,0.577591,0.126214,0.627451,0.000710,Zoo Digital Publishing,0,...,0,0,0,1,0,0,0,0,0,0
1,Madden NFL 12,Electronic Arts,0.002322,30-Aug-11,0.633512,0.174757,0.706373,0.002958,EA Tiburon,0,...,0,0,0,0,0,0,0,0,1,0
2,The Sims 2: Pets,Electronic Arts,0.005499,12-Jun-07,0.611765,0.106796,0.795098,0.003194,Maxis,0,...,0,0,0,0,0,0,0,1,0,0
3,Guilty Gear XX Accent Core Plus,PQube,0.000855,7-Apr-09,0.672471,0.252427,0.882353,0.000000,Arc System Works,0,...,1,0,0,0,0,0,0,0,0,0
4,WWE 2K14,Take-Two Interactive,0.008677,29-Oct-13,0.717647,0.165049,0.776471,0.006625,Yuke's,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442,D.C. III: Da Capo III,Kadokawa Games,0.000244,NaN,0.640459,0.271845,0.743417,0.038684,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1443,Street Fighter X Tekken,Capcom,0.004644,6-Mar-12,0.823529,0.398058,0.400000,0.018692,Capcom,0,...,1,0,0,0,0,0,0,0,0,0
1444,John Daly's ProStroke Golf,O-Games,0.000000,Canceled,0.517647,0.009709,0.706373,0.002958,Gusto Games,0,...,0,0,0,0,0,0,0,0,1,0
1445,Backyard Sports Football: Rookie Rush,Atari,0.000855,NaN,0.633512,0.174757,0.706373,0.002958,NaN,0,...,0,0,0,0,0,0,0,0,1,0
